# Numerical Integration and Differentiation

This notebook illustrates how to perform numerical integration and differentiation. 

There are several packages for doing this. Here, the focus is on QuadGK (https://github.com/JuliaMath/QuadGK.jl) and ForwardDiff (https://github.com/JuliaDiff/ForwardDiff.jl).

While QuadGK relies on well established ("standard") ways of numerical integration, ForwardDiff uses a more novel approach. (Calculus.jl provides similar functionality, applying a more traditional methods.)

## Load Packages

In [1]:
using QuadGK, ForwardDiff

include("printmat.jl")

println4Ps (generic function with 1 method)

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Numerical Integration

## The Pdf of the N(μ,σ²) Distribution 

In [3]:
function ϕNS(x,μ=0,σ²=1)       #pdf of N(μ,σ²), NS for non-standard
    σ = sqrt(σ²)
    z = (x - μ)/σ
    pdf = exp.(-0.5*z.^2)/(sqrt(2*pi)*σ)    
    return pdf
end

ϕNS (generic function with 3 methods)

In [4]:
x  = -3:0.1:3
xb = x[x.<=1.64]

plot1 = plot(x,ϕNS(x),color=:red,linewidth=2,legend=nothing)
plot!(xb,ϕNS(xb),color=:red,linewidth=2,legend=nothing,fill=(0,:red))
title!("pdf of N(0,1)")
xlabel!("x")
ylabel!("")
annotate!(1.75,0.25,"the area covers\n up to x=1.64")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 pdf of N(0,1) 
 
 
 x 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,364.498 48.5827,363.123 57.7953,361.349 67.0079,359.084 76.2205,356.225 85.4331,352.659 94.6457,348.26 103.858,342.897 113.071,336.436 122.283,328.744 
 131.496,319.698 140.709,309.19 149.921,297.136 159.134,283.487 168.346,268.235 177.559,251.425 186.772,233.156 195.984,213.593 205.197,192.967 214.409,171.573 
 223.622,149.771 232.835,127.972 242.047,106.633 251.26,86.2373 260.472,67.2805 269.685,50.2491 278.898,35.6006 288.11,23.7426 297.323,15.015 306.535,9.67266 
 315.748,7.87402 324.961,9.67266 334.173,15.015 343.386,23.7426 352.598,35.6006 361.811,50.2491 371.024,67.2805 380.236,86.2373 389.449,106.633 398.661,127.972 
 407.874,149.771 417.087,171.573 426.299,192.967 435.512,213.593 444.724,233.156 453.937,251.425 463.15,268.235 472.362,283.487 481.575,297.136 490.787,309.19 
 500,319.698 509.213,328.744 518.425,336.436 527.638,342.897 536.85,348.26 546.063,352.659 555.276,356.225 564.488,359.084 573.701,361.349 582.913,363.123 
 592.126,364.498 
 "/>
<polygon clip-path="url(#clip02)" points="
39.3701,364.498 48.5827,363.123 57.7953,361.349 67.0079,359.084 76.2205,356.225 85.4331,352.659 94.6457,348.26 103.858,342.897 113.071,336.436 122.283,328.744 
 131.496,319.698 140.709,309.19 149.921,297.136 159.134,283.487 168.346,268.235 177.559,251.425 186.772,233.156 195.984,213.593 205.197,192.967 214.409,171.573 
 223.622,149.771 232.835,127.972 242.047,106.633 251.26,86.2373 260.472,67.2805 269.685,50.2491 278.898,35.6006 288.11,23.7426 297.323,15.015 306.535,9.67266 
 315.748,7.87402 324.961,9.67266 334.173,15.015 343.386,23.7426 352.598,35.6006 361.811,50.2491 371.024,67.2805 380.236,86.2373 389.449,106.633 398.661,127.972 
 407.874,149.771 417.087,171.573 426.299,192.967 435.512,213.593 444.724,233.156 453.937,251.425 463.15,268.235 463.15,368.504 453.937,368.504 444.724,368.504 
 435.512,368.504 426.299,368.504 417.087,368.504 407.874,368.504 398.661,368.504 389.449,368.504 380.236,368.504 371.024,368.504 361.811,368.504 352.598,368.504 
 343.386,368.504 334.173,368.504 324.961,368.504 315.748,368.504 306.535,368.504 297.323,368.504 288.11,368.504 278.898,368.504 269.685,368.504 260.472,368.504 
 251.26,368.504 242.047,368.504 232.835,368.504 223.622,368.504 214.409,368.504 205.197,368.504 195.984,368.504 186.772,368.504 177.559,368.504 168.346,368.504 
 159.134,368.504 149.921,368.504 140.709,368.504 131.496,368.504 122.283,368.504 113.071,368.504 103.858,368.504 94.6457,368.504 85.4331,368.504 76.2205,368.504 
 67.0079,368.504 57.7953,368.504 48.5827,368.504 39.3701,368.504 
 " fill="#ff0000" fill-opacity="1"/>
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,364.498 48.5827,363.123 57.7953,361.349 67.0079,359.084 76.2205,356.225 85.4331,352.659 94.6457,348.26 103.858,342.897 113.071,336.436 122.283,328.744 
 131.496,319.698 140.709,309.19 149.921,297.136 159.134,283.487 168.346,268.235 177.559,251.425 186.772,233.156 195.984,213.593 205.197,192.967 214.409,171.573 
 223.622,149.771 232.835,127.972 242.047,106.633 251.26,86.2373 260.472,67.2805 269.685,50.2491 278.898,35.6006 288.11,23.7426 297.323,15.015 306.535,9.67266 
 315.748,7.87402 324.961,9.67266 334.173,15.015 343.386,23.7426 352.598,35.6006 361.811,50.2491 371.024,67.2805 380.236,86.2373 389.449,106.633 398.661,127.972 
 407.874,149.771 417.087,171.573 426.299,192.967 435.512,213.593 444.724,233.156 453.937,251.425 463.15,268.235 
 "/>
 
 the area covers 
 
 
 up to x=1.64

## Calculating Prob(x<=1.64)

In [5]:
cdf164, = QuadGK.quadgk(x->ϕNS(x),-Inf,1.64)
printlnPs("\nPr(x<=1.64) according to N(0,1): $(round(cdf164,3))")

printlnPs("\n...yes, there is a smarter way to do this calculations, but it's still a good illustration")


Pr(x<=1.64) according to N(0,1): 0.949

...yes, there is a smarter way to do this calculations, but it's still a good illustration


# Numerical Derivatives

Numerical derivatives can be calculated by a crude finite difference (see NumDer() below) or the much more sophisticated routines in the ForwardDiff.jl package. 

The function you want to differentiate must typically have only one argument (even if it is a vector). To turn a multi-argument function fn1(x,a) into a one-argument function transform it to an anonymous function by

```
x->fn1(x,a),
```
assuming that ```a``` has a value already. We can now differentiate wrt. ```x```.

In [6]:
function NumDer(fun,b0,h)           #crude function for a centered numerical derivative
    bminus = b0 - h
    bplus  = b0 + h
    hh     = bplus - bminus
    fplus  = fun(bplus)
    fminus = fun(bminus)
    D      = (fplus-fminus)/hh
    return D
end

NumDer (generic function with 1 method)

In [7]:
function fn1(x,a)                 #a simple function, to be differentiated
   return (x-1.1)^2 - a                
end

fn1 (generic function with 1 method)

In [8]:
x0 = 2

dydx_A = NumDer(x->fn1(x,0.5),x0,0.01)           #differentiate fn1(x,0.5) at x = x0
dydx_B = ForwardDiff.derivative(x->fn1(x,0.5),x0)

println("The derivative at x=$x0 is (from two different methods): ")
printmat([dydx_A dydx_B])

The derivative at x=2 is (from two different methods): 
     1.800     1.800



In [9]:
x = collect(-3:6/99:6)          #calculate the derivative at many points

dydx_A = [NumDer(x->fn1(x,0.5),x[i],0.01) for i=1:length(x)]        

dydx_B = [ForwardDiff.derivative(x->fn1(x,0.5),x[i]) for i=1:length(x)]

println("now lets plot this")

now lets plot this


In [10]:
plot(x,fn1.(x,0.5),color=:black,label="fn1()")
plot!(x,dydx_A,color=:red,line=(:dot,4),label="crude derivative")
plot!(x,dydx_B,color=:blue,line=(:dash,2),label="ForwardDiff pkg ")
title!("fn1() and its derivative")
xlabel!("x")
ylabel!("")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 fn1() and its derivative 
 
 
 x 
 
<polyline clip-path="url(#clip02)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,87.1309 43.1049,92.7939 46.8398,98.3725 50.5746,103.867 54.3094,109.277 58.0443,114.603 61.7791,119.844 65.5139,125.001 69.2488,130.074 72.9836,135.062 
 76.7185,139.966 80.4533,144.786 84.1881,149.521 87.923,154.172 91.6578,158.739 95.3926,163.221 99.1275,167.619 102.862,171.933 106.597,176.162 110.332,180.307 
 114.067,184.368 117.802,188.344 121.536,192.236 125.271,196.044 129.006,199.767 132.741,203.406 136.476,206.961 140.211,210.431 143.946,213.817 147.68,217.119 
 151.415,220.336 155.15,223.469 158.885,226.518 162.62,229.482 166.355,232.362 170.089,235.158 173.824,237.869 177.559,240.496 181.294,243.039 185.029,245.497 
 188.764,247.871 192.498,250.161 196.233,252.366 199.968,254.487 203.703,256.524 207.438,258.476 211.173,260.344 214.907,262.128 218.642,263.827 222.377,265.442 
 226.112,266.973 229.847,268.419 233.582,269.781 237.316,271.059 241.051,272.252 244.786,273.361 248.521,274.385 252.256,275.326 255.991,276.182 259.725,276.953 
 263.46,277.641 267.195,278.244 270.93,278.762 274.665,279.197 278.4,279.547 282.134,279.812 285.869,279.994 289.604,280.091 293.339,280.103 297.074,280.032 
 300.809,279.876 304.544,279.635 308.278,279.311 312.013,278.902 315.748,278.408 319.483,277.831 323.218,277.168 326.953,276.422 330.687,275.591 334.422,274.676 
 338.157,273.677 341.892,272.593 345.627,271.425 349.362,270.173 353.096,268.836 356.831,267.415 360.566,265.91 364.301,264.32 368.036,262.646 371.771,260.888 
 375.505,259.045 379.24,257.118 382.975,255.107 386.71,253.011 390.445,250.831 394.18,248.567 397.914,246.218 401.649,243.785 405.384,241.268 409.119,238.666 
 412.854,235.98 416.589,233.21 420.323,230.355 424.058,227.416 427.793,224.392 431.528,221.285 435.263,218.093 438.998,214.816 442.732,211.456 446.467,208.011 
 450.202,204.481 453.937,200.868 457.672,197.17 461.407,193.387 465.142,189.52 468.876,185.569 472.611,181.534 476.346,177.414 480.081,173.21 483.816,168.922 
 487.551,164.549 491.285,160.092 495.02,155.551 498.755,150.925 502.49,146.215 506.225,141.421 509.96,136.542 513.694,131.579 517.429,126.532 521.164,121.4 
 524.899,116.184 528.634,110.883 532.369,105.499 536.103,100.03 539.838,94.4763 543.573,88.8386 547.308,83.1166 551.043,77.3102 554.778,71.4195 558.512,65.4444 
 562.247,59.3851 565.982,53.2413 569.717,47.0133 573.452,40.7009 577.187,34.3042 580.921,27.8232 584.656,21.2578 588.391,14.6081 592.126,7.87402 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:4; stroke-opacity:1; fill:none" stroke-dasharray="2, 4" points="
 39.3701,368.504 43.1049,367.112 46.8398,365.721 50.5746,364.329 54.3094,362.938 58.0443,361.546 61.7791,360.155 65.5139,358.763 69.2488,357.372 72.9836,355.98 
 76.7185,354.589 80.4533,353.197 84.1881,351.806 87.923,350.414 91.6578,349.023 95.3926,347.631 99.1275,346.24 102.862,344.848 106.597,343.457 110.332,342.065 
 114.067,340.674 117.802,339.282 121.536,337.891 125.271,336.499 129.006,335.108 132.741,333.716 136.476,332.325 140.211,330.933 143.946,329.542 147.68,328.15 
 151.415,326.759 155.15,325.367 158.885,323.976 162.62,322.584 166.355,321.193 170.089,319.801 173.824,318.41 177.559,317.018 181.294,315.627 185.029,314.235 
 188.764,312.844 192.498,311.452 196.233,310.061 199.968,308.669 203.703,307.278 207.438,305.886 211.173,304.495 214.907,303.103 218.642,301.712 222.377,300.32 
 226.112,298.929 229.847,297.537 233.582,296.146 237.316,294.754 241.051,293.363 244.786,291.971 248.521,290.58 252.256,289.188 255.991,287.797 259.725,286.405 
 263.46,285.014 267.195,283.622 270.93,282.231 274.665,280.839 278.4,279.448 282.134,278.056 285.869,276.664 289.604,275.273 293.339,273.881 297.074,272.49 
 300.809,271.098 304.544,269.707 

## (extra) Comments on Using the ForwardDiff Package

The ForwardDiff package applies an interesting approach to calculate derivatives, using a special number type ("dual numbers"). This means that your code must be able to handle such numbers. In most cases, that is not a problem, but you may have to watch out if you create arrays to store (intermediate?) results inside the function. See the examples below

In [11]:
function fnDoesNotWork(b,a)
    z = zeros(length(b))              #will not work with ForwardDiff, since
    for i = 1:length(z)               #z cannot store dual numbers
        z[i] = b[i]*i
    end    
    return sum(z) + a
end    

function fnDoesWork(b,a)
    z = zeros(eltype(b),length(b))   #will work with ForwardDiff, since
    for i = 1:length(z)              #when b is a dual number, so is z 
        z[i] = b[i]*i
    end    
    return sum(z) + a
end

fnDoesWork (generic function with 1 method)

In [12]:
b0 = [1.5;2]

try
    ForwardDiff.gradient(b->fnDoesNotWork(b,1),b0)
    println("using fnDoesNotWork(): ",dydx)
catch    
    dydx = ForwardDiff.gradient(b->fnDoesWork(b,1),b0)
    println("using fnDoesWork(): ",dydx)    
end

using fnDoesWork(): [1.0, 2.0]
